![]()

# `mdh` R-R-R Manipulator

Kevin J. Walchko, Phd

xxx 2021

---

Using `mdh` we can calculate a 3R manipulator to find both
the forward kinematics (fk) and the inverse kinematics (ik)
of the manipulator. The 3R is common for many type of robotics
applications like arms and legs.

Looking at a 3d arm, we can break it up into a top-down look
at the desired position ($[x_D, y_D, z_D]$) composed of the
first joint and a rotated (90 degrees) plane containing the
remtaining 2 joints that look like a simplifed 2D arm.

![](ik-3r.png)

The rotation matrix which matches the world frame to the the
end effector is:

$$
R^T_W = 
\begin{bmatrix}
    cos(\phi) & -sin(\phi) & 0 \\
    sin(\phi) & cos(\phi) & 0 \\
    0 & 0 & 1
\end{bmatrix}
$$

Thus to get the angle of the arm to satisfy a given point,
you need to solve these equations:

$$
r = R^T_W \\
a_1 = L_1 \\
a_2 = L_2 \\
a_3 = L_3 \\
\theta_1 + \theta_2 + \theta_3 = atan2(r_{21}, r_{11}) \\
a_1 c_1 + a_2 c_{12} + a_3 c_{123} = x \\
a_1 s_1 + a_2 s_{12} a_3 s_{123} = y
$$

You can use gradient decent or the optimization capability of
`scipy` to solve these.

### Analytic

The 2D look:

![](ik-2r.png)

![](ik-2rb.png)

## Algorithm for 3R Inverse Kinematics 

$$
q_1 = atan2(y_D, x_D) \\
q_1' = atan2(y_D, x_D) + \pi \\
(q_2,q3) = RIK2(L_2, L_3, - \sqrt{x_D^2 +y_D^2}, -z_D+L_1)
$$

```python
def RIK2(L1, L2, x, y):
    """
    L1: first link length
    L2: second link length
    x: desired x
    y: desired y
    """
    xx = sqrt(x*x + y*y)
    c = (xx - L1*L1 - L2*L2)/(2*L1*L2)
    if abs(c) > 1:
        return 0
    elif c == 1:
        return (atan2(y, x), 0,)
    elif c == -1 and xx > 0:
        return (atan2(y,x), pi,)
    elif c == -1 and xx == 0:
        return None
    else:
        q2 = acos(c)
        q22 = -acos(c)
        t = atan2(y,x)
        q1,q11 = None, None
        for q, qq in zip([q1,q11],[q2,q22]):
            q = t - atan2(L2*sin(qq), L1 + L2*cos(qq))
        return ((q1,q2), (q11,q22))
```

## References

- [Stochastic Gradient Decent with Numpy](https://realpython.com/gradient-descent-algorithm-python/)
- UI: [Inverse Kinematics](http://motion.cs.illinois.edu/RoboticSystems/InverseKinematics.html)

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
from numpy.testing import assert_allclose
from numpy import pi

from matplotlib import pyplot as plt

In [4]:
import mdh
from mdh.kinematic_chain import KinematicChain
from mdh import UnReachable # exception

In [53]:
# modified DH parameters:
#     alpha a theta d in radians
#     L1, L2, L3 in cm
# types: revolute=1, prismatic=2 (not implemented yet)

class R3:
    """
    Let's make an interface for the robot arm shown above.
    Using the mdh can be confusing, since there are more
    joints and links than maybe be thought of.
    """
    def __init__(self, L1, L2, L3):

        dh = [
            {'alpha': 0,  'a': 0, 'theta': 0, 'd': L1, 'type': 1},
            {'alpha': pi/2, 'a': 0, 'theta': 0, 'd': 0, 'type': 1},
            {'alpha': 0, 'a': L2, 'theta': 0, 'd': 0, 'type': 1},
            {'alpha': 0, 'a': L3, 'theta': 0, 'd': 0, 'type': 1},
        ]

        self.kc = KinematicChain.from_parameters(dh)
        
    def __str__(self):
        s = []
        for l in self.kc:
            s.append(str(l))
        return "\n".join(s)
    
    def __iter__(self):
        for i in self.kc:
            yield i
    
    def fk(self, t1, t2, t3):
        angles = np.deg2rad([t1, 0, t2, t3])
        return self.kc.forward(angles)
    
    def ik(self, x, y, z):
        return self.kc.inverse((x,y,z,))
    
kc = R3(10,20,20)
print(kc)

Rev[deg]: alpha:  0.0 a:  0.0 theta:  0.0 d: 10.0
Rev[deg]: alpha: 90.0 a:  0.0 theta:  0.0 d:  0.0
Rev[deg]: alpha:  0.0 a: 20.0 theta:  0.0 d:  0.0
Rev[deg]: alpha:  0.0 a: 20.0 theta:  0.0 d:  0.0


In [54]:
# forward kinematics
print(f"{kc.fk(0,0,0)}")

[[ 1.  0.  0. 40.]
 [ 0.  0. -1.  0.]
 [ 0.  1.  0. 10.]
 [ 0.  0.  0.  1.]]


In [55]:
# forward kinematics
print(f"{kc.fk(-45,0,0)}")

[[  0.707   0.     -0.707  28.284]
 [ -0.707   0.     -0.707 -28.284]
 [  0.      1.      0.     10.   ]
 [  0.      0.      0.      1.   ]]


In [56]:
for l in kc:
    print(l)

Rev[deg]: alpha:  0.0 a:  0.0 theta:  0.0 d: 10.0
Rev[deg]: alpha: 90.0 a:  0.0 theta:  0.0 d:  0.0
Rev[deg]: alpha:  0.0 a: 20.0 theta:  0.0 d:  0.0
Rev[deg]: alpha:  0.0 a: 20.0 theta:  0.0 d:  0.0


In [58]:
# inverse kinematics
deg = kc.ik(40,0,10)
rad = np.rad2deg(deg)
print(f">> {rad}")

ValueError: operands could not be broadcast together with shapes (4,4) (3,) 